In [1]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels import regression,stats
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta

In [2]:
ib=IB()
util.startLoop()
ib.connect('127.0.0.1',7497, clientId= 65)

<IB connected to 127.0.0.1:7497 clientId=65>

## 4th April

Priority is pulling the data before implementing correlation and cointegration.

Meet again on Wednesday after pulling data. Split approaches between IB and other sources. 
Yahoo finance - directly download as CSV
Backtesting period?
Need live information for trading

Hardcode Asian ADR List

Still need to figure out IB live trading

1. Manually intensive process to pull the tickers (~70)
2. Figuring out pairs to trade - check stock pairs that have converged in the past since there may not guarantee of convergence


## Data Cleaning

Possible functions to implement: 
1. get_data - We are currently scraping from Yahoo finance, but we still need to get historical data from IB
2. clean_data (remove any nan)
3. functions to sync up the dates?
4. Adjust forex?

Format for all the Historical Data:

Combine the underlying stock adjusted close + look at the foreign exchange close

Write another Jupyter Notebook dedicated to data cleaning and pulling

In [4]:
import pandas_datareader as pdr
from datetime import datetime
tickers = ['BARC.L', 'BCS']
for ticker in tickers:
    df =  pdr.get_data_yahoo(symbols=ticker, start=datetime(1990, 1, 1), end=datetime(2020, 1, 1))
    df.to_csv(ticker+'.csv')

In [5]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1990-01-02,6.130678,6.027814,6.130678,6.048387,135495.0,1.904180
1990-01-03,6.130678,6.089533,6.130678,6.110105,159191.0,1.923610
1990-01-04,6.151251,6.089533,6.151251,6.089533,21874.0,1.917134
1990-01-05,6.192396,6.110105,6.130678,6.192396,503700.0,1.949518
1990-01-08,6.274687,6.212969,6.233542,6.274687,283749.0,1.975424
1990-01-09,6.336406,6.254115,6.336406,6.254115,315344.0,1.968948
1990-01-10,6.315833,6.171824,6.315833,6.171824,44355.0,1.943040
1990-01-11,6.233542,6.151251,6.212969,6.151251,46178.0,1.936563
1990-01-12,6.089533,6.027814,6.089533,6.027814,91140.0,1.897702


In [3]:
def get_data(contract, history, freq, side, endDateTime = ''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime=endDateTime,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    df = util.df(bar)
    return df 

In [9]:
ticker = '7203'
secType = 'STK'
exchange = 'TSEJ'
currency = 'JPY'

contract = Contract(symbol = ticker, secType = secType, exchange = exchange,currency = currency)
ib.qualifyContracts(contract)

history = '1 Y'
freq = '1 day'
side ='Trades'
endDate = '20191231 23:59:59'

px = get_data (contract, history, freq, side, endDate)

Error 162, reqId 34: Historical Market Data Service error message:No market data permissions for TSEJ STK, contract: Contract(secType='STK', conId=13905804, symbol='7203', exchange='TSEJ', primaryExchange='TSEJ', currency='JPY', localSymbol='7203.T', tradingClass='7203')
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
[WinError 10054] An existing connection was forcibly closed by the remote host


In [4]:
ticker = 'BCS'
secType = 'STK'
exchange = 'NYSE'
currency = 'USD'

contract = Contract(symbol = ticker, secType = secType, exchange = exchange,currency = currency)
ib.qualifyContracts(contract)

history = '1 Y'
freq = '1 day'
side ='Trades'
endDate = '20191231 23:59:59'

px = get_data (contract, history, freq, side, endDate)

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: jfarm; usfarm; euhmds; ushmds.nj; hkhmds; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: jfarm; usfarm; euhmds; ushmds.nj; hkhmds; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1

In [26]:
ticker = 'BARC'
secType = 'STK'
exchange = 'LSE'
currency = 'GBP'

contract = Contract(symbol = ticker, secType = secType, exchange = exchange,currency = currency)
ib.qualifyContracts(contract)

order = MarketOrder("BUY",10)
ib.placeOrder(contract,order)


ConnectionError: Not connected

In [23]:
ticker = 'BARC'
secType = 'STK'
# exchange = 'LSE'
currency = 'GBP'

contract = Contract(symbol = ticker, secType = secType, currency = currency)
ib.qualifyContracts(contract)

history = '1 Y'
freq = '1 day'
side ='Trades'
endDate = '20191231 23:59:59'

px = get_data (contract, history, freq, side, endDate)

Ambiguous contract: Contract(secType='STK', symbol='BARC', currency='GBP'), possibles are [Contract(secType='STK', conId=908940, symbol='BARC', exchange='SMART', primaryExchange='LSE', currency='GBP', localSymbol='BARC', tradingClass='BARC'), Contract(secType='STK', conId=908940, symbol='BARC', exchange='LSE', primaryExchange='LSE', currency='GBP', localSymbol='BARC', tradingClass='BARC'), Contract(secType='STK', conId=908940, symbol='BARC', exchange='CHIXUK', primaryExchange='LSE', currency='GBP', localSymbol='BARCl', tradingClass='BARC'), Contract(secType='STK', conId=908940, symbol='BARC', exchange='TRQXUK', primaryExchange='LSE', currency='GBP', localSymbol='BARCl', tradingClass='BARC'), Contract(secType='STK', conId=908940, symbol='BARC', exchange='BATEUK', primaryExchange='LSE', currency='GBP', localSymbol='BARCl', tradingClass='BARC')]
Error 321, reqId 29: Error validating request.-'bT' : cause - Please enter exchange, contract: Contract(secType='STK', symbol='BARC', currency='G

## Strategy

Strategy implementable as a class? Strategy helper functions can be written here too.

Possible functions to implement:
1. on_data()
2. hedge ratio?
3. risk management - VaR, stoploss?

Account for slippage and illiquidity?
Market Order vs Limit Order

In [ ]:
def on_data():

## Backtesting

Should keep track of all trades done, equity at each point of time 

Should log each test by writing into a file

Keep track of current cash

Keep track of slippage/illiquidity here?

## Live Trading

Get market data at regular intervals
Update metrics to decide whether to make trade

Get portfolio information as well: 
equity, margin, etc